In [1]:
import matplotlib.pyplot as plt
import pybaseball as pyb
import pandas as pd
import requests
from bs4 import BeautifulSoup

from pybaseball import statcast_pitcher
from pybaseball import playerid_lookup

In [2]:
kersh = statcast_pitcher('2015-01-01', '2022-12-31', 477132)
kersh = kersh[['player_name','pitcher','game_year','pitch_type','pitch_name',
               'release_speed','effective_speed','release_pos_x','release_pos_z',
               'release_extension','release_spin_rate','spin_axis','zone']]


Gathering Player Data


c:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\pybaseball\cache\cache.py:58: DtypeWarning: Columns (43) have mixed types.Specify dtype option on import or set low_memory=False.
  result = func(*args, **kwargs)


In [3]:
kersh_summary = kersh.groupby(['game_year', 'pitch_type']).mean()


In [14]:
url = "https://baseballsavant.mlb.com/savant-player/clayton-kershaw-477132?stats=statcast-r-pitching-mlb"
soup = BeautifulSoup(requests.get(url).content, 'html5lib')

In [5]:
# Get pitch movement data
pitch_movement = soup.find_all("table", {"id": "pitchMovement"})[0].contents[3].contents

pitch_movement_data = []
for row in pitch_movement:
    if isinstance(row, str):
        continue

    pitch_movement_data.append([record.contents[0] for record in row.find_all("span") if record.contents])

pitch_movement_data = pd.DataFrame(pitch_movement_data, columns=['year','pitch','hand','count','speed',
                                                               'vertical_movement','vertical_movement_vs_avg','vertical_movement_pct_vs_avg',
                                                               'horizontal_movement','horizontal_movement_vs_avg','horizontal_movement_pct_vs_avg'])


In [9]:
# Get run value data
run_values = soup.find_all("table", {"id": "runValues"})[0].contents[3].contents

run_values_data = []
for row in run_values:
    if isinstance(row, str):
        continue

    record_data = []
    record_data.append(row.contents[1].contents[0])

    count = 0
    for record in row.find_all("span"):
        if count == 2:
            record_data.append(row.contents[9].contents[0])
        
        if record.contents:
            record_data.append(record.contents[0])
        
        count += 1

    run_values_data.append(record_data)

run_values_data = pd.DataFrame(run_values_data, columns=['year','pitch','rv_100','run_value','count','usage',
                                                         'PA','BA','SLG','wOBA','whiff_pct','k_pct','put_away_pct',
                                                         'xBA','xSLG','xwOBA','hard_hit_pct'])


In [7]:
# Get spin direction data
spin_axis = soup.find_all("table", {"id": "spinAxis"})[0].contents[3].contents

spin_axis_data = []
for row in spin_axis:
    if isinstance(row, str):
        continue

    record_data = []
    record_data.append(row.contents[1].contents[0])

    for record in row.find_all("span"):
        if record.contents:
            record_data.append(record.contents[0])

    spin_axis_data.append(record_data)

spin_axis_data = pd.DataFrame(spin_axis_data, columns=['year','pitch','count','speed','active_spin_pct',
                                                       'total_movement','spin_based','observed','deviation'])
